In [1]:
import pandas as pd
import numpy as np
from siuba import *
from calitp import *

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
bc = to_snakecase(
    pd.read_excel(
        "gs://calitp-analytics-data/data-analyses/grants/Grant+Projects_3_17_2022.xlsx"
    )
)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [5]:
bc.funding_program.value_counts()

5310 Trad          964
Section 5311       590
CARES Act          166
CRRSAA             153
5310 Exp           151
Section 5311(f)    109
5339 (State)        95
Toll Credits        52
ARPA                50
5339 (National)     48
CARES Act (F)       43
5311(f) Cont        41
CMAQ (FTA 5311)     31
5311(f) Round 2     27
Name: funding_program, dtype: int64

In [6]:
df = bc>>filter(_.funding_program.str.contains('5310'))


In [7]:
len(df>>count(_.organization_name))

147

In [8]:
df.sample()

grant_fiscal_year funding_program                grant_number  \
480               2017       5310 Trad  CA-2017-169 | 0017000186-T   

     project_year      organization_name        upin  \
480          2017  North Valley Services  BCG0000654   

                           description     ali contract_number  \
480  Purchase Replacement < 30 Ft Bus   111204    64AC18-00650   

     allocationamount  encumbered_amount  expendedamount  activebalance  \
480           73000.0            73000.0        71440.58        1559.42   

     closedoutbalance project_status  project_closed_by  project_closed_date  \
480                 0           Open                NaN                  NaN   

     project_closed_time  
480                  NaN

In [9]:
df.project_year.value_counts()

2019    604
2017    507
2018      4
Name: project_year, dtype: int64

In [26]:
(
    df
    >> filter(_.organization_name.str.contains("Dignity"))
    >> group_by(_.grant_number, _.funding_program)
    >> count(_.organization_name)
)

grant_number funding_program  \
0  CA-2017-169 | 0017000186-E        5310 Exp   
1  CA-2017-169 | 0017000186-E        5310 Exp   
2  CA-2017-169 | 0017000186-T       5310 Trad   
3  CA-2017-169 | 0017000186-T       5310 Trad   
4  CA-2020-244 | 0020000273-T       5310 Trad   

                 organization_name   n  
0           Dignity Community Care   1  
1  Dignity Health Connected Living   1  
2           Dignity Community Care  10  
3  Dignity Health Connected Living   6  
4  Dignity Health Connected Living   4

In [27]:
##same grantnumber, different names

In [31]:
df>>group_by(_.grant_number)>>summarize(unique_orgs = _.organization_name.nunique())

grant_number  unique_orgs
0  CA-2017-169 | 0017000186-E           57
1  CA-2017-169 | 0017000186-T           93
2  CA-2020-244 | 0020000273-E           50
3  CA-2020-244 | 0020000273-T           74

In [30]:
df>>group_by(_.organization_name)>>count(_.grant_number)

organization_name                grant_number   n
0                        ARC Bakersfield  CA-2017-169 | 0017000186-T  20
1                    ARC Imperial Valley  CA-2017-169 | 0017000186-E   2
2                    ARC Imperial Valley  CA-2020-244 | 0020000273-E   2
3                    ARC Imperial Valley  CA-2020-244 | 0020000273-T   4
4                        Able Industries  CA-2017-169 | 0017000186-T   6
..                                   ...                         ...  ..
269          Wilshire Community Services  CA-2017-169 | 0017000186-E   1
270          Wilshire Community Services  CA-2020-244 | 0020000273-E   1
271                 Work Training Center  CA-2017-169 | 0017000186-T   7
272  Yolo County Transportation District  CA-2017-169 | 0017000186-E   2
273  Yolo County Transportation District  CA-2017-169 | 0017000186-T   2

[274 rows x 3 columns]

In [10]:
from shared_utils import geography_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You m

In [11]:
org = geography_utils.aggregate_by_geography(
    df,
    group_cols=["organization_name"],
    nunique_cols=["grant_fiscal_year", "grant_number", "contract_number"],
    count_cols=["upin"],
    sum_cols=[
        "allocationamount",
        "encumbered_amount",
        "expendedamount",
        "activebalance",
        "closedoutbalance",
    ],
    mean_cols=[
        "allocationamount",
        "encumbered_amount",
        "expendedamount",
        "activebalance",
        "closedoutbalance",
    ],
    rename_cols=True,
)

In [12]:
org

organization_name  activebalance_sum  \
0                                       Amador Transit         -268853.00   
1                                     Angel View, Inc.           25729.61   
2                                  ARC Imperial Valley          -44628.20   
3                               Area 1 Agency on Aging               0.00   
4    Asian Community Center of Sacramento Valley, I...          541809.50   
..                                                 ...                ...   
142                          Self-Help for the Elderly          242098.00   
143                                    The Respite Inn           56000.00   
144                          Ukiah Senior Center, Inc.           56000.00   
145  United Cerebral Palsy Association Inc of Stani...         -239007.28   
146                          Valley Achievement Center          230100.00   

     allocationamount_sum  closedoutbalance_sum  encumbered_amount_sum  \
0                469995.0                     0               738848.0   
1                554995.0                     0               774990.0   
2                549178.0                     0               865145.0   
3                146534.0                     0               146534.0   
4               1425169.0                     0              2320371.0   
..                    ...                   ...                    ...   
142              242098.0                     0               484196.0   
143               56000.0                     0               112000.0   
144               56000.0                     0               112000.0   
145              284800.0                     0               569600.0   
146              230100.0                     0               460200.0   

     expendedamount_sum  activebalance_mean  allocationamount_mean  \
0             738848.00        -67213.25000             117498.750   
1             529265.39          3216.20125              69374.375   
2             593806.20         -5578.52500              68647.250   
3             146534.00             0.00000             146534.000   
4             883359.50         27090.47500              71258.450   
..                  ...                 ...                    ...   
142                0.00         30262.25000              30262.250   
143                0.00         28000.00000              28000.000   
144                0.00         28000.00000              28000.000   
145           523807.28        -29875.91000              35600.000   
146                0.00         38350.00000              38350.000   

     closedoutbalance_mean  encumbered_amount_mean  expendedamount_mean  \
0                        0              184712.000         184712.00000   
1                        0               96873.750          66158.17375   
2                        0              108143.125          74225.77500   
3                        0              146534.000         146534.00000   
4                        0              116018.550          44167.97500   
..                     ...                     ...                  ...   
142                      0               60524.500              0.00000   
143                      0               56000.000              0.00000   
144                      0               56000.000              0.00000   
145                      0               71200.000          65475.91000   
146                      0               76700.000              0.00000   

     upin_count  contract_number_nunique  grant_fiscal_year_nunique  \
0             4                        3                          2   
1             8                        4                          2   
2             8                        3                          2   
3             1                        1                          1   
4            20                        5                          2   
..          ...                      ...               

In [13]:
org["organization_name"] = (
    org["organization_name"]
    .str.replace(",", "")
    .str.replace(".", "")
    .str.replace("/", " ")
    #.str.replace("\\", "")
)

/tmp/ipykernel_983/2494736175.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [14]:
org_name= org>>select(_.organization_name)

In [15]:
org_name

organization_name
0                                       Amador Transit
1                                       Angel View Inc
2                                  ARC Imperial Valley
3                               Area 1 Agency on Aging
4    Asian Community Center of Sacramento Valley In...
..                                                 ...
142                          Self-Help for the Elderly
143                                    The Respite Inn
144                            Ukiah Senior Center Inc
145  United Cerebral Palsy Association Inc of Stani...
146                          Valley Achievement Center

[147 rows x 1 columns]

In [15]:
#pd.set_option("display.max_colwidth", 100)

In [16]:
org_name>>filter(_.organization_name.str.contains('/'))

Empty DataFrame
Columns: [organization_name]
Index: []

In [17]:
org_name>>filter(_.organization_name.str.contains('Community Bridges'))

organization_name
12  Community Bridges Liftline

## Read in Airtable Organization Information

In [21]:
itp= to_snakecase(pd.read_csv("gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/organizations_All_Organizations.csv"))


In [22]:
itp.sample()

name organization_type roles record_creation_time ntp_id  \
310  City of San Marcos         City/Town   NaN    1/10/2022 10:00pm    NaN   

     itp_id  opm_id_drmt  dotid brand alias details website  \
310     NaN          NaN    NaN   NaN   NaN     NaN     NaN   

    parent_organization administrating_organization mobility_services_managed  \
310                 NaN                         NaN  San Marcos Catch a Ride!   

    missing_static funding_sources_for_managed_transportation  \
310            NaN                                     public   

    mobility_services_operated gtfs_datasets_produced  \
310                        NaN                    NaN   

    service_type__from_mobility_services_managed_  \
310                                on-demand,NEMT   

    currently_operating__from_mobility_services_managed_  \
310                                 1 checked out of 1     

    currently_operating__from_mobility_services_operated_  \
310                                                NaN      

    service_type__from_mobility_services_operated_ headquarters_place  \
310                                            NaN         San Marcos   

    funding_programs  total_voms__ntd_  service_area_sq_miles__ntd_  \
310              NaN               NaN                          NaN   

     service_area_population__ntd_ caltrans_district mpo_rtpa  \
310                            NaN               NaN      NaN   

    planning_authority tracking_category reporting_category assist_category  \
310                NaN               NaN                NaN             NaN   

    eligibility_programs gtfs_datasets  \
310                  NaN           NaN   

    gtfs_dataset__from_mobility_services_managed_ fare_systems  \
310                                           NaN          NaN   

    gtfs_schedule_status fares_v2_status               flex_status  \
310                  NaN             NaN  Needed - Special service   

      services_needing_alerts  \
310  San Marcos Catch a Ride!   

    services_needing_tripupdates_or_vehiclepositions  \
310                         San Marcos Catch a Ride!   

     #_of_fixed_route_services  #_services_w__complete_rt_status  \
310                          0                                 0   

     #_fixed_route_services_w__static_gtfs  \
310                                      0   

     complete_static_gtfs_coverage__1=yes_  complete_rt_coverage  \
310                                      1                     1   

     >=1_gtfs_feed_for_any_service__1=yes_  >=_1_complete_rt_set__1=yes_  \
310                                      0                             0   

    service_availability_category__from_mobility_services_managed_  \
310                                        Conditional               

     #_fixed_route_or_deviated_fixed_route_services  \
310                                               0   

     #_fixed_route_or_deviated_fixed_route_service_w__static_gtfs  \
310                                                  0              

     #_services_with_missing_static_feed_for_fixed_route_or_deviated_fixed_route  \
310                                                  0                             

    county_geography county_geography_3 gtfs_static_status  \
310              NaN                NaN          Static OK   

    gtfs_realtime_status service_availability service___component  
310        RT Incomplete          Conditional                 NaN

In [23]:
itp = itp >> select(
    _.name,
    _.organization_type,
    _.roles,
    _.ntp_id,
    _.itp_id,
    _.alias,
    _.opm_id_drmt,
    _.mobility_services_operated,
    _.mpo_rtpa,
)

In [24]:
itp['name'] = itp['name'].str.replace(',','').str.replace('.','').str.replace('/',' ')
#.str.replace("\\", "")

/tmp/ipykernel_989/2080570801.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [25]:
itp>>select(_.name)

name
0                                        A-Paratransit
1                                          ABC Shuttle
2                                             Able Inc
3                                          Abrazar Inc
4                                      Access Services
..                                                 ...
823       Yosemite Area Regional Transportation System
824                      Yuba-Sutter Transit Authority
825  Yuma County Intergovernmental Public Transport...
826                                        Yurok Tribe
827                                          Zonar Inc

[828 rows x 1 columns]

In [26]:
itp.rename(columns = {'ntp_id':'ntd_id'}, inplace=True)

## Read in NTD Agency Info

In [27]:
ntd = to_snakecase(pd.read_excel("gs://calitp-analytics-data/data-analyses/5311 /2020_Agency_Information.xlsx"))

In [28]:
ntd = ntd>>filter(_.state=='CA')

In [29]:
ntd = ntd >> select(
    _.ntd_id,
    _.agency_name,
    _.doing_business_as,
    _.reporter_acronym,
    _.doing_business_as,
    _.organization_type,
    _.reporter_type,
    _.reported_by_ntd_id,
    _.city,
)

In [30]:
ntd

ntd_id                                    agency_name  \
884   90003  San Francisco Bay Area Rapid Transit District   
885   90004                 Golden Empire Transit District   
887   90006       Santa Cruz Metropolitan Transit District   
888   90007                               City of Modesto    
889   90008                           City of Santa Monica   
...     ...                                            ...   
2869  A0008                          Kern Regional Transit   
2870  A0009                     Humboldt Transit Authority   
2952  A0013         Metropolitan Transportation Commission   
2955  A0016                    Redding Area Bus Authority    
3003  A0022     San Luis Obispo Regional Transit Authority   

         doing_business_as reporter_acronym  \
884                    NaN             BART   
885                    NaN              GET   
887                    NaN            SCMTD   
888   Modesto Area Express              MAX   
889           Big Blue Bus              NaN   
...                    ...              ...   
2869                   NaN               KT   
2870                   NaN              HTA   
2952                   NaN              MTC   
2955                   NaN             RABA   
3003                   NaN           SLORTA   

                                      organization_type       reporter_type  \
884   Independent Public Agency or Authority of Tran...       Full Reporter   
885   Independent Public Agency or Authority of Tran...       Full Reporter   
887   Independent Public Agency or Authority of Tran...       Full Reporter   
888   City, County or Local Government Unit or Depar...       Full Reporter   
889   City, County or Local Government Unit or Depar...       Full Reporter   
...                                                 ...                 ...   
2869  City, County or Local Government Unit or Depar...  Group Plan Sponsor   
2870  Independent Public Agency or Authority of Tran...  Group Plan Sponsor   
2952                  MPO, COG or Other Planning Agency  Group Plan Sponsor   
2955  Independent Public Agency or Authority of Tran...  Group Plan Sponsor   
3003  Independent Public Agency or Authority of Tran...  Group Plan Sponsor   

     reported_by_ntd_id             city  
884                 NaN          Oakland  
885                 NaN      Bakersfield  
887                 NaN       Santa Cruz  
888                 NaN          Modesto  
889                 NaN     Santa Monica  
...                 ...              ...  
2869                NaN      Bakersfield  
2870                NaN           Eureka  
2952                NaN    San Francisco  
2955                NaN          Redding  
3003                NaN  San Luis Obispo  

[241 rows x 8 columns]

In [31]:
ntd>>filter(_.agency_name.str.contains("Calaveras"))

ntd_id               agency_name doing_business_as reporter_acronym  \
2829  9R02-99442  Calaveras Transit Agency               NaN              NaN   

                                      organization_type   reporter_type  \
2829  Independent Public Agency or Authority of Tran...  Rural Reporter   

     reported_by_ntd_id         city  
2829               9R02  San Andreas

In [32]:
ntd['agency_name'] = ntd['agency_name'].str.replace(',','').str.replace('.','').str.replace('/',' ')
#.str.replace("\\", "")

/tmp/ipykernel_989/1640863875.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


## Merge NTD & ITP info

#### On Name

In [33]:
m = pd.merge(ntd, itp, left_on="agency_name", right_on="name", how="outer", indicator=True)

In [34]:
m._merge.value_counts()

right_only    649
both          181
left_only      60
Name: _merge, dtype: int64

In [35]:
m.sample(5)

ntd_id_x          agency_name doing_business_as reporter_acronym  \
162    99268  Bishop Paiute Tribe               NaN              NaN   
676      NaN                  NaN               NaN              NaN   
335      NaN                  NaN               NaN              NaN   
331      NaN                  NaN               NaN              NaN   
652      NaN                  NaN               NaN              NaN   

    organization_type_x     reporter_type reported_by_ntd_id    city  \
162               Tribe  Reduced Reporter                NaN  Bishop   
676                 NaN               NaN                NaN     NaN   
335                 NaN               NaN                NaN     NaN   
331                 NaN               NaN                NaN     NaN   
652                 NaN               NaN                NaN     NaN   

                                               name      organization_type_y  \
162                             Bishop Paiute Tribe                      NaN   
676  Norman Y Mineta San Jose International Airport                      NaN   
335                                 City of Belmont                City/Town   
331                           City of Arroyo Grande                City/Town   
652                       Milestones of Development  Non-Profit Organization   

    roles ntd_id_y  itp_id alias  opm_id_drmt  \
162   NaN    99268    40.0   NaN          NaN   
676   NaN      NaN   225.0   SJC          NaN   
335   NaN      NaN     NaN   NaN          NaN   
331   NaN      NaN     NaN   NaN          NaN   
652   NaN      NaN   440.0   NaN          NaN   

               mobility_services_operated mpo_rtpa      _merge  
162                                   NaN      NaN        both  
676                                   NaN      NaN  right_only  
335  Belmont Twin Pines Senior Center Van      NaN  right_only  
331                                   NaN      NaN  right_only  
652             Milestones of Development      NaN  right_only

In [36]:
nomatch1 = m >> filter(_._merge == "left_only")>>select(_.ntd_id_x,
                                                        _.agency_name,
                                                       _.doing_business_as, 
                                                        _.reporter_acronym,
                                                        _.organization_type_x,
                                                       _.reporter_type,
                                                       _.reported_by_ntd_id,
                                                       _.city)

In [37]:
nomatch2= m >> filter(_._merge == "right_only")>>select(_.name,
                                                        _.organization_type_y,
                                                        _.roles,
                                                        _.ntd_id_y,
                                                        _.itp_id, 
                                                        _.alias,
                                                        _.opm_id_drmt,
                                                        _.mobility_services_operated,
                                                        _.mpo_rtpa)

In [38]:
nomatch1>>filter(_.ntd_id_x==90260)

ntd_id_x        agency_name                    doing_business_as  \
116    90260  City of Compton    Compton Renaissance Transit Service   

    reporter_acronym                                organization_type_x  \
116              CRT  City, County or Local Government Unit or Depar...   

        reporter_type reported_by_ntd_id     city  
116  Reduced Reporter                NaN  Compton

In [39]:
nomatch2>>filter(_.ntd_id_y=='90260')

name organization_type_y roles ntd_id_y  itp_id alias  \
354  City of Compton           City/Town   NaN    90260    77.0   NaN   

     opm_id_drmt mobility_services_operated  \
354          NaN        Compton Dial-A-Ride   

                                           mpo_rtpa  
354  Southern California Association of Governments

In [40]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x  

In [41]:
nomatch1['ntd_id_x'] = nomatch1['ntd_id_x'].apply(get_num)
nomatch2['ntd_id_y'] = nomatch2['ntd_id_y'].apply(get_num)

In [42]:
nomatch1.sample()

ntd_id_x                                        agency_name  \
128    90273  Los Angeles County Dept of Public Works - Flor...   

    doing_business_as reporter_acronym  \
128               NaN           LACDPW   

                                   organization_type_x     reporter_type  \
128  City, County or Local Government Unit or Depar...  Reduced Reporter   

    reported_by_ntd_id      city  
128                NaN  Alhambra

In [43]:
nomatch_merge = pd.merge(nomatch1, nomatch2, left_on="doing_business_as", right_on="name", how="outer", indicator=True)

In [44]:
to_add1 = nomatch_merge>>filter(_._merge=='both')

In [45]:
to_add1

ntd_id_x         agency_name            doing_business_as reporter_acronym  \
51    90236  Stanislaus County   Stanislaus Regional Transit            StaRT   

                                  organization_type_x     reporter_type  \
51  City, County or Local Government Unit or Depar...  Reduced Reporter   

   reported_by_ntd_id     city                         name  \
51                NaN  Modesto  Stanislaus Regional Transit   

   organization_type_y roles ntd_id_y  itp_id alias  opm_id_drmt  \
51  Independent Agency   NaN    90236   325.0   NaN    1050003.0   

                           mobility_services_operated  \
51  Stanislaus Regional Transit,StaRT Commuter,Med...   

                             mpo_rtpa _merge  
51  Stanislaus Council of Governments   both

In [46]:
nomatch_merge2 = pd.merge(nomatch1, nomatch2, left_on="ntd_id_x", right_on="ntd_id_y", how="outer", indicator=True)

In [47]:
# no matches
to_add2 = nomatch_merge2>>filter(_._merge=='both')
to_add2

ntd_id_x                                        agency_name  \
0        90007                                   City of Modesto    
1        90022                                   City of Norwalk    
2        90024                                 City of La Mirada    
3        90029                                          Omnitrans   
4        90035                        Gold Coast Transit District   
6        90089                                  County of Sonoma    
7        90091                                  City of Visalia     
8        90092                       City of Fairfield California   
9        90144        Livermore   Amador Valley Transit Authority   
10       90154  Los Angeles County Metropolitan Transportation...   
12       90157                                  Access Services     
13       90162         The Eastern Contra Costa Transit Authority   
14       90182                          Altamont Corridor Express   
15       90196                                   County of Placer   
16       90206         San Luis Obispo Regional Transit Authority   
18       90223                                    Paratransit Inc   
19       90225  San Francisco Bay Area Water Emergency Transpo...   
20       90236                                 Stanislaus County    
21       90260                                  City of Compton     
22       90265                                  City of El Monte    
34       90283                           City of Manhattan Beach    
35       90298                                  County of Ventura   
36       90301                                  City of Lakewood    
43  9R02-91048                                 County of Siskiyou   
44  9R02-91057                            Tuolumne County Transit   
45  9R02-91059                              Kern Regional Transit   
48  9R02-91088                              Glenn Transit Service   
49  9R02-91093                                    City of Eureka    
50  9R02-91095                     Nevada County Transit Services   
52  9R02-99438        County of Shasta Department of Public Works   

                           doing_business_as reporter_acronym  \
0                       Modesto Area Express              MAX   
1                    Norwalk Transit System               NTS   
2                         La Mirada Transit               LMT   
3                                        NaN              NaN   
4                                        NaN             GCTD   
6                      Sonoma County Transit              SCT   
7                            Visalia Transit               VT   
8               Fairfield and Suisun Transit             FAST   
9                                        NaN            LAVTA   
10                                     Metro           LACMTA   
12                                       NaN              NaN   
13                         Tri Delta Transit              NaN   
14                                       NaN              ACE   
15  Placer County Department of Public Works              DPW   
16                                       NaN           SLORTA   
18                                       NaN              NaN   
19                   San Francisco Bay Ferry             WETA   
20               Stanislaus Regional Transit            StaRT   
21       Compton Renaissance Transit Service              CRT   
22  City of El Monte Transportation Division              NaN   
34                                       NaN              CMB   
35                                       NaN            PWATD   
36                              DASH Transit             DASH   
43                   Siskiyou County Transit            STAGE   
44                                       NaN              TCT   
45                                       NaN               KT   
48                                       NaN              GTS   
49                    Eureka Transit Service  

In [48]:
len(to_add2)

30

In [49]:
nomatch_merge = pd.merge(nomatch1, nomatch2, left_on="reporter_acronym", right_on="alias", how="outer", indicator=True)

In [50]:
to_add3 = nomatch_merge>>filter(_._merge=='both')
to_add4 = (to_add3>>filter(_.alias.notnull()))
to_add4

ntd_id_x                                  agency_name  \
0           90007                             City of Modesto    
12879       90092                 City of Fairfield California   
12880       90144  Livermore   Amador Valley Transit Authority   
12883       90182                    Altamont Corridor Express   
12885       90206   San Luis Obispo Regional Transit Authority   
12886       A0022   San Luis Obispo Regional Transit Authority   
12902  9R02-91048                           County of Siskiyou   

                  doing_business_as reporter_acronym  \
0              Modesto Area Express              MAX   
12879  Fairfield and Suisun Transit             FAST   
12880                           NaN            LAVTA   
12883                           NaN              ACE   
12885                           NaN           SLORTA   
12886                           NaN           SLORTA   
12902       Siskiyou County Transit            STAGE   

                                     organization_type_x       reporter_type  \
0      City, County or Local Government Unit or Depar...       Full Reporter   
12879  City, County or Local Government Unit or Depar...       Full Reporter   
12880  Independent Public Agency or Authority of Tran...       Full Reporter   
12883  Independent Public Agency or Authority of Tran...       Full Reporter   
12885  Independent Public Agency or Authority of Tran...       Full Reporter   
12886  Independent Public Agency or Authority of Tran...  Group Plan Sponsor   
12902  City, County or Local Government Unit or Depar...      Rural Reporter   

      reported_by_ntd_id             city  \
0                    NaN          Modesto   
12879                NaN        Fairfield   
12880                NaN        Livermore   
12883                NaN         Stockton   
12885                NaN  San Luis Obispo   
12886                NaN  San Luis Obispo   
12902               9R02            Yreka   

                                                 name organization_type_y  \
0                                     City of Modesto           City/Town   
12879                               City of Fairfield           City/Town   
12880  Livermore   Amador\n  Valley Transit Authority  Independent Agency   
12883            San Joaquin Regional Rail Commission  Independent Agency   
12885     San Luis Obispo Regional Transit Authority   Independent Agency   
12886     San Luis Obispo Regional Transit Authority   Independent Agency   
12902                                 Siskiyou County              County   

      roles    ntd_id_y  itp_id   alias  opm_id_drmt  \
0       NaN       90007   203.0     MAX    1050002.0   
12879   NaN       90092   110.0    FAST     448002.0   
12880   NaN       90144   167.0   LAVTA     401003.0   
12883   NaN       90182    10.0     ACE    1039003.0   
12885   NaN       90206   289.0  SLORTA     540005.0   
12886   NaN       90206   289.0  SLORTA     540005.0   
12902   NaN  9R02-91048    83.0   STAGE     247002.0   

                              mobility_services_operated  \
0      Modesto Area Express,eTrans Dial-A-Ride,Modest...   
12879  Fairfield and Suisun Transit,DART Paratransit,...   
12880  Tri-Valley Wheels,Tri-Valley Wheels Paratransi...   
12883                          Altamont Corridor Express   
12885  South County Area Transit,San Luis Obispo Regi...   
12886  South County Area Transit,San Luis Obispo Regi...   
12902               Siskiyou Transit and General Express   

                                     mpo_rtpa _merge  
0           Stanislaus Council of Governments   both  
12879  Metropolitan Transportation Commission   both  
12880  Metropolitan Transportation Commission   both  
12883      San Joaquin Council of Governments   both  
12885  San Luis Obispo Council of Governments   both  
12886  San Luis Obispo Council of Governments   both  
12902                                     NaN   both

In [51]:
## Subsetting original merge

In [52]:
match = m>>filter(_._merge=='both')

In [53]:
match.sample()

ntd_id_x                              agency_name doing_business_as  \
192  9R02-91038  Sierra County Transportation Commission               NaN   

    reporter_acronym                                organization_type_x  \
192             SCTC  City, County or Local Government Unit or Depar...   

      reporter_type reported_by_ntd_id         city  \
192  Rural Reporter               9R02  Downieville   

                                        name organization_type_y  \
192  Sierra County Transportation Commission                 NaN   

                                       roles    ntd_id_y  itp_id alias  \
192  Regional Transportation Planning Agency  9R02-91038   304.0   NaN   

     opm_id_drmt mobility_services_operated mpo_rtpa _merge  
192          NaN                        NaN      NaN   both

In [54]:
matches = pd.concat([match, to_add1], ignore_index=True, sort=False)

In [55]:
matches = pd.concat([matches, to_add2], ignore_index=True, sort=False)

In [56]:
matches = pd.concat([matches, to_add4], ignore_index=True, sort=False)

In [57]:
matches.sample()


ntd_id_x        agency_name       doing_business_as reporter_acronym  \
51    90163  City of Camarillo  Camarillo Area Transit              CAT   

                                  organization_type_x     reporter_type  \
51  City, County or Local Government Unit or Depar...  Reduced Reporter   

   reported_by_ntd_id       city               name organization_type_y roles  \
51                NaN  Camarillo  City of Camarillo           City/Town   NaN   

   ntd_id_y  itp_id alias  opm_id_drmt  \
51    90163    54.0   NaN     756002.0   

                           mobility_services_operated  \
51  Camarillo Area Transit,Camarillo Trolly,Camari...   

                                          mpo_rtpa _merge  
51  Southern California Association of Governments   both

In [58]:
len(matches)

219

* exporting to manipualte data 


In [59]:
#matches.to_csv('matches.csv')

In [60]:
#m.to_csv('ntd_itp_merge.csv')

## Merge Matches with 5310

* first fill in the missing information

In [61]:
matches>>filter(_.reporter_acronym.isnull())>>filter(_.alias.notnull())

ntd_id_x                               agency_name doing_business_as  \
8         90014     Alameda-Contra Costa Transit District        AC Transit   
12        90019      Sacramento Regional Transit District     Sacramento RT   
22        90042                           City of Gardena            GTrans   
24        90044                           City of Arcadia   Arcadia Transit   
48        90156                   City of San Luis Obispo       SLO Transit   
107       90287  Palos Verdes Peninsula Transit Authority               NaN   
131       99423                          City of Glendale               NaN   
171  9R02-91112              Colusa County Transit Agency               NaN   
175  9R02-99454          Palo Verde Valley Transit Agency               NaN   
210  9R02-91095            Nevada County Transit Services               NaN   

    reporter_acronym                                organization_type_x  \
8                NaN  Independent Public Agency or Authority of Tran...   
12               NaN  Independent Public Agency or Authority of Tran...   
22               NaN  City, County or Local Government Unit or Depar...   
24               NaN  City, County or Local Government Unit or Depar...   
48               NaN  City, County or Local Government Unit or Depar...   
107              NaN  Independent Public Agency or Authority of Tran...   
131              NaN  City, County or Local Government Unit or Depar...   
171              NaN  City, County or Local Government Unit or Depar...   
175              NaN  Independent Public Agency or Authority of Tran...   
210              NaN  City, County or Local Government Unit or Depar...   

        reporter_type reported_by_ntd_id                    city  \
8       Full Reporter                NaN                 Oakland   
12      Full Reporter                NaN              Sacramento   
22      Full Reporter                NaN                 Gardena   
24   Reduced Reporter                NaN                 Arcadia   
48      Full Reporter                NaN         San Luis Obispo   
107  Reduced Reporter                NaN  Palos Verdes Peninsula   
131     Full Reporter                NaN                Glendale   
171    Rural Reporter               9R02                  Colusa   
175    Rural Reporter               9R02                  Blythe   
210    Rural Reporter               9R02             Nevada City   

                                         name organization_type_y roles  \
8       Alameda-Contra Costa Transit District  Independent Agency   NaN   
12       Sacramento Regional Transit District  Independent Agency   NaN   
22                            City of Gardena           City/Town   NaN   
24                            City of Arcadia           City/Town   NaN   
48                    City of San Luis Obispo           City/Town   NaN   
107  Palos Verdes Peninsula Transit Authority  Independent Agency   NaN   
131                          City of Glendale           City/Town   NaN   
171              Colusa County Transit Agency  Independent Agency   NaN   
175          Palo Verde Valley Transit Agency  Independent Agency   NaN   
210                             Nevada County              County   NaN   

       ntd_id_y  itp_id                    alias  opm_id_drmt  \
8         90014     4.0               AC Transit     401001.0   
12        90019   273.0      SacRT,Sacramento RT     334004.0   
22          NaN   118.0                   GTrans     719008.0   
24        90044    17.0          Arcadia Transit     719003.0   
48        90156   287.0                SLO Trans     540004.0   
107       90287   239.0        Desert Roadrunner     719029.0   
131       99423   120.0                  Beeline          NaN   
171  9R02-91112    74.0                     CCTA     306002.0   
175         NaN   238.0  PVVTA,Desert Roadrunner     833005.0   
210  9R02-91095   221.0       Gold Country Stage     329002.0   

                    

In [62]:
matches.reporter_acronym.fillna(matches['alias'], inplace=True)

In [63]:
compare_col = np.where(matches["reporter_acronym"] == matches["alias"], True, False)
matches["compare_acronym"] = compare_col
matches.compare_acronym.value_counts()

False    184
True      35
Name: compare_acronym, dtype: int64

In [64]:
matches>>filter(_.compare_acronym==False)>>filter(_.reporter_acronym.notnull())>>filter(_.alias.notnull())>>select(_.agency_name, _.name, _.reporter_acronym, _.alias)

agency_name  \
6                San Joaquin Regional Transit District   
7          Santa Clara Valley Transportation Authority   
9                     City and County of San Francisco   
13         Santa Barbara Metropolitan Transit District   
15               San Diego Metropolitan Transit System   
16                                      City of Fresno   
35              Metropolitan Transportation Commission   
36              Metropolitan Transportation Commission   
40               Peninsula Corridor Joint Powers Board   
43                                 City of Los Angeles   
187                                  County of Sonoma    
191  Los Angeles County Metropolitan Transportation...   

                                                  name reporter_acronym  \
6                San Joaquin Regional Transit District              RTD   
7          Santa Clara Valley Transportation Authority              VTA   
9                     City and County of San Francisco            SFMTA   
13         Santa Barbara Metropolitan Transit District            SBMTD   
15               San Diego Metropolitan Transit System              MTS   
16                                      City of Fresno              FAX   
35              Metropolitan Transportation Commission              MTC   
36              Metropolitan Transportation Commission              MTC   
40               Peninsula Corridor Joint Powers Board            PCJPB   
43                                 City of Los Angeles            LADOT   
187                                      Sonoma County              SCT   
191  Los Angeles County Metropolitan Transportation...           LACMTA   

                                                 alias  
6                                                SJRTD  
7                                                SCVTA  
9                                   Muni,SF Muni,SFMTA  
13   "Santa Barbara MTD\trecswCrw6a6htmXJ4\t0\t\tMo...  
15                                               SDMTD  
16                             Fresno Area Express,FAX  
35                     MTC,Bay Area Metro,Bay Area 511  
36                     MTC,Bay Area Metro,Bay Area 511  
40                                      Caltrain,PCJPB  
43                              LA DOT,Los Angeles DOT  
187                                 Healdsburg Shuttle  
191                                              LAMTA

* Only one not marching is San Diego Metropolitan Transit System, but MTS should be the right name
* Others are close

In [65]:
matches = matches.drop(columns={'alias'})

In [66]:
matches>>filter(_.organization_type_x.isnull())>>filter(_.organization_type_y.notnull())

Empty DataFrame
Columns: [ntd_id_x, agency_name, doing_business_as, reporter_acronym, organization_type_x, reporter_type, reported_by_ntd_id, city, name, organization_type_y, roles, ntd_id_y, itp_id, opm_id_drmt, mobility_services_operated, mpo_rtpa, _merge, compare_acronym]
Index: []

In [67]:
matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   ntd_id_x                    219 non-null    object  
 1   agency_name                 219 non-null    object  
 2   doing_business_as           74 non-null     object  
 3   reporter_acronym            139 non-null    object  
 4   organization_type_x         218 non-null    object  
 5   reporter_type               219 non-null    object  
 6   reported_by_ntd_id          53 non-null     object  
 7   city                        219 non-null    object  
 8   name                        219 non-null    object  
 9   organization_type_y         200 non-null    object  
 10  roles                       10 non-null     object  
 11  ntd_id_y                    200 non-null    object  
 12  itp_id                      209 non-null    float64 
 13  opm_id_drmt         

In [68]:
matches>>filter(_.roles.notnull())

ntd_id_x                                      agency_name  \
35        90094           Metropolitan Transportation Commission   
36        A0013           Metropolitan Transportation Commission   
37        90095             San Diego Association of Governments   
52        90164         Ventura County Transportation Commission   
66        90208          Butte County Association of Governments   
117       90297           San Luis Obispo Council of Governments   
121       90303  Santa Barbara County Association of Governments   
150  9R02-91038          Sierra County Transportation Commission   
172  9R02-91116    Alpine County Local Transportation Commission   
173  9R02-91119          Plumas County Transportation Commission   

    doing_business_as reporter_acronym  \
35                NaN              MTC   
36                NaN              MTC   
37                NaN           SANDAG   
52                NaN             VCTC   
66                NaN             BCAG   
117               NaN           SLOCOG   
121               NaN            SBCAG   
150               NaN             SCTC   
172               NaN              NaN   
173               NaN             PCTC   

                                   organization_type_x       reporter_type  \
35                   MPO, COG or Other Planning Agency       Full Reporter   
36                   MPO, COG or Other Planning Agency  Group Plan Sponsor   
37                   MPO, COG or Other Planning Agency       Full Reporter   
52   Independent Public Agency or Authority of Tran...       Full Reporter   
66                   MPO, COG or Other Planning Agency       Full Reporter   
117                  MPO, COG or Other Planning Agency    Reduced Reporter   
121                  MPO, COG or Other Planning Agency    Reduced Reporter   
150  City, County or Local Government Unit or Depar...      Rural Reporter   
172  City, County or Local Government Unit or Depar...      Rural Reporter   
173  City, County or Local Government Unit or Depar...      Rural Reporter   

    reported_by_ntd_id             city  \
35                 NaN    San Francisco   
36                 NaN    San Francisco   
37                 NaN        San Diego   
52                 NaN        Camarillo   
66                 NaN            Chico   
117                NaN  San Luis Obispo   
121                NaN    Santa Barbara   
150               9R02      Downieville   
172               9R02     Markleeville   
173               9R02           Quincy   

                                                name     organization_type_y  \
35            Metropolitan Transportation Commission                MPO/RTPA   
36            Metropolitan Transportation Commission                MPO/RTPA   
37              San Diego Association of Governments                     NaN   
52          Ventura County Transportation Commission                     NaN   
66           Butte County Association of Governments     Joint Powers Agency   
117           San Luis Obispo Council of Governments  Council of Governments   
121  Santa Barbara County Association of Governments                     NaN   
150          Sierra County Transportation Commission                     NaN   
172    Alpine County Local Transportation Commission                     NaN   
173          Plumas County Transportation Commission                     NaN   

                                                 roles    ntd_id_y  itp_id  \
35                  Metropolitan Planning Organization         NaN   200.0   
36                  Metropolitan Planning Organization         NaN   200.0   
37   Metropolitan Planning Organization,Regional Tr...       90095   276.0   
52             Regional Transportation Planning Agency       90164   380.0   
66   Regional Transportation Planning Agency,Metrop...       90208    47.0   
117  Regional Transportation Planning Agency,Metrop...       90297   288.0   
121  Metropolitan Planning Organiza

In [69]:
# def recategorize(df):   
#     if (df.roles.str.contains('Metropolitan Planning Organization')):
#         return 'MPO/RTPA'
#     elif (df.roles.str.contains('Regional Transportation Planning Agency')) :
#         return 'MPO/RTPA'
#     elif (df.roles.str.contains('Regional Transportation Planning Agency,Metropolitan Planning Organization')):
#         return 'MPO/RTPA'
#     else: 
#         return NaN

In [70]:
#matches['organization_type_z'] = matches.apply(recategorize, axis = 1)

In [71]:
matches.organization_type_y.fillna(matches['roles'], inplace=True)

In [72]:
matches.sample()

ntd_id_x                              agency_name doing_business_as  \
173  9R02-91119  Plumas County Transportation Commission               NaN   

    reporter_acronym                                organization_type_x  \
173             PCTC  City, County or Local Government Unit or Depar...   

      reporter_type reported_by_ntd_id    city  \
173  Rural Reporter               9R02  Quincy   

                                        name  \
173  Plumas County Transportation Commission   

                         organization_type_y  \
173  Regional Transportation Planning Agency   

                                       roles ntd_id_y  itp_id  opm_id_drmt  \
173  Regional Transportation Planning Agency      NaN     NaN          NaN   

    mobility_services_operated mpo_rtpa _merge  compare_acronym  
173                        NaN      NaN   both            False

In [73]:
matches>>filter(_.ntd_id_x.isnull())>>filter(_.ntd_id_y.notnull())

Empty DataFrame
Columns: [ntd_id_x, agency_name, doing_business_as, reporter_acronym, organization_type_x, reporter_type, reported_by_ntd_id, city, name, organization_type_y, roles, ntd_id_y, itp_id, opm_id_drmt, mobility_services_operated, mpo_rtpa, _merge, compare_acronym]
Index: []

In [74]:
matches = matches.drop(columns={'ntd_id_y'})
matches = matches.drop(columns={'reported_by_ntd_id'})


In [75]:
matches = matches.rename(columns={'ntd_id_x':'ntd_id'})

In [76]:
matches.sample(3)

ntd_id         agency_name            doing_business_as reporter_acronym  \
65   90205   City of Elk Grove                       e-tran              NaN   
181  90236  Stanislaus County   Stanislaus Regional Transit            StaRT   
102  90281     City of Lynwood                          NaN              COL   

                                   organization_type_x     reporter_type  \
65   City, County or Local Government Unit or Depar...     Full Reporter   
181  City, County or Local Government Unit or Depar...  Reduced Reporter   
102  City, County or Local Government Unit or Depar...  Reduced Reporter   

          city                         name organization_type_y roles  itp_id  \
65   Elk Grove            City of Elk Grove           City/Town   NaN   105.0   
181    Modesto  Stanislaus Regional Transit  Independent Agency   NaN   325.0   
102    Lynwood              City of Lynwood           City/Town   NaN   186.0   

     opm_id_drmt                         mobility_services_operated  \
65           NaN                                                NaN   
181    1050003.0  Stanislaus Regional Transit,StaRT Commuter,Med...   
102          NaN                                                NaN   

                                           mpo_rtpa _merge  compare_acronym  
65                                              NaN   both            False  
181               Stanislaus Council of Governments   both            False  
102  Southern California Association of Governments   both            False

* wondering if `opm_id_drmt` and `ali` match up

In [77]:
compare_col2 = np.where(matches["agency_name"] == matches["name"], True, False)
matches["compare_names"] = compare_col2
matches.compare_names.value_counts()

True     181
False     38
Name: compare_names, dtype: int64

In [78]:
matches>>filter(_.compare_names==False)>>select(_.agency_name, _.name)

agency_name  \
181                                 Stanislaus County    
182                                   City of Modesto    
183                                   City of Norwalk    
184                                 City of La Mirada    
185                                          Omnitrans   
186                        Gold Coast Transit District   
187                                  County of Sonoma    
188                                  City of Visalia     
189                       City of Fairfield California   
190        Livermore   Amador Valley Transit Authority   
191  Los Angeles County Metropolitan Transportation...   
192                                  Access Services     
193         The Eastern Contra Costa Transit Authority   
194                          Altamont Corridor Express   
195                                   County of Placer   
196         San Luis Obispo Regional Transit Authority   
197                                    Paratransit Inc   
198  San Francisco Bay Area Water Emergency Transpo...   
199                                 Stanislaus County    
200                                  City of Compton     
201                                  City of El Monte    
202                           City of Manhattan Beach    
203                                  County of Ventura   
204                                  City of Lakewood    
205                                 County of Siskiyou   
206                            Tuolumne County Transit   
207                              Kern Regional Transit   
208                              Glenn Transit Service   
209                                    City of Eureka    
210                     Nevada County Transit Services   
211        County of Shasta Department of Public Works   
212                                   City of Modesto    
213                       City of Fairfield California   
214        Livermore   Amador Valley Transit Authority   
215                          Altamont Corridor Express   
216         San Luis Obispo Regional Transit Authority   
217         San Luis Obispo Regional Transit Authority   
218                                 County of Siskiyou   

                                                  name  
181                        Stanislaus Regional Transit  
182                                    City of Modesto  
183                                    City of Norwalk  
184                                  City of La Mirada  
185                                          OmniTrans  
186                                 Gold Coast Transit  
187                                      Sonoma County  
188                                    City of Visalia  
189                                  City of Fairfield  
190     Livermore   Amador\n  Valley Transit Authority  
191  Los Angeles County Metropolitan Transportation...  
192                                    Access Services  
193             Eastern Contra Costa Transit Authority  
194               San Joaquin Regional Rail Commission  
195                                      Placer County  
196        San Luis Obispo Regional Transit Authority   
197                                   Paratransit Inc   
198  San Francisco Bay Area Water Emergency Transit...  
199                        Stanislaus Regional Transit  
200                                    City of Compton  
201                                   City of El Monte  
202                            City of Manhattan Beach  
203                                     Ventura County  
204                                   City of Lakewood  
205                                    Siskiyou County  
206                     Tuolumne County Transit Agency  
207                                        Kern County  
208                                       Glenn County  
209                                     City of Eureka  
210                                      Nevada County  
211          

## Merging 5310 Org names to CW

In [79]:
org_name.sample()

organization_name
27  Independent Living Partnership

In [80]:
matches.sample()

ntd_id       agency_name doing_business_as reporter_acronym  \
42  90146  Foothill Transit               NaN              NaN   

                                  organization_type_x  reporter_type  \
42  Independent Public Agency or Authority of Tran...  Full Reporter   

           city              name organization_type_y roles  itp_id  \
42  West Covina  Foothill Transit  Independent Agency   NaN   112.0   

    opm_id_drmt mobility_services_operated  \
42     719007.0           Foothill Transit   

                                          mpo_rtpa _merge  compare_acronym  \
42  Southern California Association of Governments   both            False   

    compare_names  
42           True

In [81]:
m1 = (
    pd.merge(
        org_name,
        matches,
        left_on="organization_name",
        right_on="agency_name",
        how="outer",
        indicator="in_5310",
    )
)

In [82]:
len(m1>>filter(_.in_5310=='both'))

28

In [83]:
_5310_matches1 = m1>>filter(_.in_5310=='both')

In [84]:
_5310_matches1.sample(5)

organization_name      ntd_id  \
49                          Town of Truckee  9R02-91101   
43        San Mateo County Transit District       90009   
93   Metropolitan Transportation Commission       90094   
54      Yolo County Transportation District       90090   
102                               Omnitrans       90029   

                                agency_name doing_business_as  \
49                          Town of Truckee               NaN   
43        San Mateo County Transit District          SamTrans   
93   Metropolitan Transportation Commission               NaN   
54      Yolo County Transportation District           Yolobus   
102                               Omnitrans               NaN   

    reporter_acronym                                organization_type_x  \
49               TOT  City, County or Local Government Unit or Depar...   
43               NaN  Independent Public Agency or Authority of Tran...   
93               MTC                  MPO, COG or Other Planning Agency   
54              YCTD  Independent Public Agency or Authority of Tran...   
102              NaN  Independent Public Agency or Authority of Tran...   

      reporter_type            city                                    name  \
49   Rural Reporter         Truckee                         Town of Truckee   
43    Full Reporter      San Carlos       San Mateo County Transit District   
93    Full Reporter   San Francisco  Metropolitan Transportation Commission   
54    Full Reporter        Woodland     Yolo County Transportation District   
102   Full Reporter  San Bernardino                               OmniTrans   

    organization_type_y                               roles  itp_id  \
49            City/Town                                 NaN   340.0   
43   Independent Agency                                 NaN   290.0   
93             MPO/RTPA  Metropolitan Planning Organization   200.0   
54   Independent Agency                                 NaN   372.0   
102  Independent Agency                                 NaN   232.0   

     opm_id_drmt                         mobility_services_operated  \
49      329003.0  TART Connect,Truckee TART Dial A Ride,Placer T...   
43      441010.0  SamTrans,Redi-Wheels,RediCoast,Bayshore Brisba...   
93           NaN                                                NaN   
54      357002.0        Yolobus,Yolobus Special,Causeway Connection   
102     836004.0        OmniTrans,OmniRide,OmniTrans sbX,OmniAccess   

                                           mpo_rtpa _merge compare_acronym  \
49                                              NaN   both           False   
43           Metropolitan Transportation Commission   both           False   
93                                              NaN   both           False   
54           Sacramento Area Council of Governments   both           False   
102  Southern California Association of Governments   both           False   

    compare_names in_5310  
49           True    both  
43           True    both  
93           True    both  
54           True    both  
102         False    both

* merging on the other agency name to see if there are any more matches

In [85]:
_5310_matches2= (
    pd.merge(
        org_name,
        matches,
        left_on="organization_name",
        right_on="name",
        how="outer",
        indicator="in_5310",
    )
)

In [86]:
_5310_matches2.in_5310.value_counts()

right_only    191
left_only     121
both           28
Name: in_5310, dtype: int64

* same # of matches

In [87]:
#checking another name column

In [88]:
(
    pd.merge(
        org_name,
        matches,
        left_on="organization_name",
        right_on="doing_business_as",
        how="outer",
        indicator="in_5310",
    )
).in_5310.value_counts()

right_only    219
left_only     147
both            0
Name: in_5310, dtype: int64

#### Checking to see what is left for the agencies 

In [89]:
_5310_left = (m1>>filter(_.in_5310=='left_only'))

In [90]:
_5310_left = _5310_left>>select(_.organization_name)

In [91]:
# merging with unmatched.

In [92]:
unmatched = (m>>filter(_._merge != 'both'))

In [93]:
unmatched.sample(2)

ntd_id_x agency_name doing_business_as reporter_acronym  \
718      NaN         NaN               NaN              NaN   
725      NaN         NaN               NaN              NaN   

    organization_type_x reporter_type reported_by_ntd_id city            name  \
718                 NaN           NaN                NaN  NaN  Presidio Trust   
725                 NaN           NaN                NaN  NaN         Ride On   

         organization_type_y roles ntd_id_y  itp_id alias  opm_id_drmt  \
718                      NaN   NaN      NaN   257.0   NaN          NaN   
725  Non-Profit Organization   NaN      NaN   465.0   NaN          NaN   

                            mobility_services_operated  \
718                                          PresidiGo   
725  Ride On Access Shuttle,Ride On Community Inter...   

                                   mpo_rtpa      _merge  
718  Metropolitan Transportation Commission  right_only  
725                                     NaN  right_only

In [94]:
(
    pd.merge(
        _5310_left,
        unmatched,
        left_on="organization_name",
        right_on="agency_name",
        how="outer",
        indicator="in_5310",
    )
).in_5310.value_counts()

right_only    709
left_only     121
both            0
Name: in_5310, dtype: int64

In [95]:
unmatch_merge2 = (
    pd.merge(
        _5310_left,
        unmatched,
        left_on="organization_name",
        right_on="name",
        how="outer",
        indicator="in_5310",
    )
)

In [96]:
unmatch_merge2.in_5310.value_counts()

right_only    672
left_only      84
both           37
Name: in_5310, dtype: int64

In [97]:
_5310_remaining2 = unmatch_merge2 >>filter(_.in_5310=='left_only')

In [98]:
unmatch_merge2 = unmatch_merge2 >>filter(_.in_5310=='both')

In [99]:
#for good measure
(
    pd.merge(
        _5310_left,
        unmatched,
        left_on="organization_name",
        right_on="doing_business_as",
        how="outer",
        indicator="in_5310",
    )
).in_5310.value_counts()

right_only    709
left_only     121
both            0
Name: in_5310, dtype: int64

#### Appending `_5310_left` to include these last matches

5310 matches from matches in ntd & itp
* _5310_matches1
* _5310_matches2

5310 matches from unmatches in ntd & itp
* unmatch_merge2

In [100]:
_5310_matches1 = _5310_matches1>>filter(_.in_5310==('both'))
_5310_matches2 = _5310_matches2>>filter(_.in_5310==('both'))

In [101]:
_5310_info = pd.concat([_5310_matches1, _5310_matches2], ignore_index=True, sort=False)

In [102]:
_5310_info = pd.concat([_5310_info, unmatch_merge2], ignore_index=True, sort=False)

In [103]:
_5310_info = _5310_info.drop_duplicates()

In [104]:
_5310_info>>count(_.organization_name)>>filter(_.n>1)

organization_name  n
11                         City of Visalia  2
21              Humboldt Transit Authority  2
29  Metropolitan Transportation Commission  2

* get rid of duplicates in downloaded version. these are single entries with double matches (on name and NTD_ids)

In [105]:
_5310_info

organization_name      ntd_id  \
0                  City of Roseville       90168   
1   Eastern Sierra Transit Authority  9R02-91062   
2             Lake Transit Authority  9R02-91053   
3      Marin County Transit District       90234   
4    Morongo Basin Transit Authority  9R02-91090   
..                               ...         ...   
88           Reach Out Morongo Basin         NaN   
89    Porterville Sheltered Workshop         NaN   
90         Self-Help for the Elderly         NaN   
91                   The Respite Inn         NaN   
92         Valley Achievement Center         NaN   

                         agency_name  doing_business_as reporter_acronym  \
0                  City of Roseville  Roseville Transit              NaN   
1   Eastern Sierra Transit Authority                NaN             ESTA   
2             Lake Transit Authority                NaN              LTA   
3      Marin County Transit District                NaN              NaN   
4    Morongo Basin Transit Authority                NaN             MBTA   
..                               ...                ...              ...   
88                               NaN                NaN              NaN   
89                               NaN                NaN              NaN   
90                               NaN                NaN              NaN   
91                               NaN                NaN              NaN   
92                               NaN                NaN              NaN   

                                  organization_type_x     reporter_type  \
0   City, County or Local Government Unit or Depar...  Reduced Reporter   
1   Independent Public Agency or Authority of Tran...    Rural Reporter   
2   Independent Public Agency or Authority of Tran...    Rural Reporter   
3   Independent Public Agency or Authority of Tran...     Full Reporter   
4   Independent Public Agency or Authority of Tran...    Rural Reporter   
..                                                ...               ...   
88                                                NaN               NaN   
89                                                NaN               NaN   
90                                                NaN               NaN   
91                                                NaN               NaN   
92                                                NaN               NaN   

           city                              name      organization_type_y  \
0     Roseville                 City of Roseville                City/Town   
1        Bishop  Eastern Sierra Transit Authority       Independent Agency   
2    Lower Lake            Lake Transit Authority       Independent Agency   
3    San Rafael     Marin County Transit District       Independent Agency   
4   Joshua Tree   Morongo Basin Transit Authority       Independent Agency   
..          ...                               ...                      ...   
88          NaN           Reach Out Morongo Basin  Non-Profit Organization   
89          NaN    Porterville Sheltered Workshop  Non-Profit Organization   
90          NaN         Self-Help for the Elderly  Non-Profit Organization   
91          NaN                   The Respite Inn  Non-Profit Organization   
92          NaN         Valley Achievement Center  Non-Profit Organization   

   roles  itp_id  opm_id_drmt  \
0    NaN   271.0     331005.0   
1    NaN    99.0     926002.0   
2    NaN   159.0     117002.0   
3    NaN   194.0     421002.0   
4    NaN   212.0     540002.0   
..   ...     ...          ...   
88   NaN   454.0          NaN   
89   NaN   452.0          NaN   
90   NaN     NaN          NaN   
91   NaN     NaN          NaN   
92   NaN   467.0          NaN   

                           mobility_services_operated  \
0             Roseville Transit,Roseville Dial-A-Ride   
1   Eastern Sierra Transit Authority Community Rou...   
2   Lake Transit,Clearlake/Lower Lake Dial-A-Ride,...   
3   Marin Transit,

In [109]:
#_5310_info.to_csv('5310_agencies_matches.csv')

* exporting to put information in for unmatched organizations

In [109]:
#_5310_remaining2.to_csv('5310_unmatched.csv')

In [ ]:
_5310info

In [ ]:
len(_5310info>>count(_.organization_name)>>arrange(-_.n)>>filter(_.n>1))

In [ ]:
_5310info = _5310info.drop_duplicates()

In [ ]:
#still have duplicates
len(_5310info>>count(_.organization_name)>>arrange(-_.n)>>filter(_.n>1))

In [ ]:
_5310info>>filter(_.organization_name=='Humboldt Transit Authority')